In [1]:
import pandas as pd
import numpy as np


import transformers
from transformers import pipeline
import torch

c:\Users\daniel.martinezcarre\AppData\Local\miniconda3\envs\deeplearning\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path_file = "C:\\Users\\daniel.martinezcarre\\Desktop\\Repositorios\\ia_repositorio\\sistemas_de_aprendizaxe_automatico\\neumonia\\peruvia-food-reviews\\reviews.csv"

data = pd.read_csv(path_file)

data.columns

Index(['id_review', 'review', 'title', 'score', 'likes', 'id_nick', 'service',
       'date', 'platform'],
      dtype='object')

### Se recogen sólo los reviews del dataframe

In [3]:
data_modelo = data[:100]
data_modelo

,id_review,review,title,score,likes,id_nick,service,date,platform
0,R1245,Muy buena presentación y servicio sin embargo ...,Muy buena presentación y servicio,3.0,0,uisahelenad2018,71036.0,1 years ago,tripadvisor
1,R1246,Desde la presentación de los platos a la calid...,Una experiencia,5.0,0,turrutia,71036.0,1 years ago,tripadvisor
2,R1247,El mejor lugar para reencontrame con mis amigo...,Felicitaciones a Statera!,5.0,0,orthStar25333985745,71036.0,1 years ago,tripadvisor
3,R1248,Excelente experiencia Comida maravillosa con e...,Aniversario de boda,5.0,0,atima J,71036.0,1 years ago,tripadvisor
4,R1249,Mi pasión es viajar y disfrutar de las grandez...,No te lo querrás perder!,5.0,0,nbocadoalmundo,71036.0,1 years ago,tripadvisor
...,...,...,...,...,...,...,...,...,...
95,R1629,"Es un rinconcito maravilloso, con unas 12 mesa...",Fin de semana,4.0,0,labbeb,76722.0,3 years ago,tripadvisor
96,R1630,Llegamos por las recomendaciones de trip y Fou...,Buen rincón,5.0,0,covas24,76722.0,3 years ago,tripadvisor
97,R1631,"Recomiendo el lugar,la cosa.ida vasca es muy b...",Muy bueno,4.0,0,orenzo C,76722.0,3 years ago,tripadvisor
98,R1632,Queríamos ir porque las opiniones eran muy bue...,Decepcionante,2.0,0,ititin,76722.0,3 years ago,tripadvisor


## Comprobar si hay una GPU disponible

In [4]:
device = 0 if torch.cuda.is_available() else -1
device

-1

In [5]:
# https://huggingface.co/docs/transformers/main_classes/logging
transformers.utils.logging.set_verbosity_error()

## Análisis sentimiento

In [6]:
from transformers import pipeline
from transformers import AutoTokenizer

# Se carga el modelo pre-entrenado de BERT para análisis de sentimiento en ESPAÑOL
nlp = pipeline("text-classification", model="nlptown/bert-base-multilingual-uncased-sentiment", device=device)
tokenize = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

In [8]:
from transformers import BertTokenizer

# Carga el tokenizador BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokeniza las reseñas
tokenized_reviews = []
for d in range(len(data_modelo)):
    review = data_modelo.iloc[d]["review"]
    encoded_inputs = tokenize(review, truncation=True, padding='max_length', max_length=800)
    tokenized_reviews.append(encoded_inputs)

### Aplicación del análisis de sentimiento a cada review

In [11]:
resultados = []
for review in tokenized_reviews:
    tokens_str = [str(token) for token in review['input_ids']]
    result = nlp(tokens_str)[0]
    resultados.append(result)

In [14]:
reales = data_modelo['score']
pred = []
for i in resultados:
    nota = int(i['label'][0])
    pred.append(nota)

print(f'Longitud reales {len(reales)}, predichos {len(pred)}')

# mae = np.,mean(np.abs(reales - pred))
# rmse 0 sqrt((reales**2-pred**2).mean())


Longitud reales 100, predichos 100


In [24]:
reales_array = np.array(reales)
pred_array = np.array(pred)

mae = np.mean(np.abs(reales_array - pred_array))
rmse = np.sqrt(((reales_array - pred_array)**2).mean())

print(f'Métricas de error\n- MSE (Mean Squared Error): {mae}\n- RMSE (Root Mean Squared Error): {rmse}')

Métricas de error
- MSE (Mean Squared Error): 0.36
- RMSE (Root Mean Squared Error): 0.8246211251235321
